# Code analysis with Langchain + Azure OpenAI + Azure Cognitive Search (vector store)

The following demo will show how to analyze your existing by using both Azure OpenAI and Search with the help of Langchain.

**LangChain** is an open-source framework that simplifies the creation of applications using large language models (LLMs). It provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications. You can use it to connect a language model to other sources of data, and allow it to interact with its environment.


In [2]:
import os
import json
import sys

#from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch

In [3]:
sys.version

'3.11.5 (main, Sep 26 2023, 13:12:46) [GCC 10.2.1 20210110]'

## Documents

In [4]:
!ls notebooks/*.*

'notebooks/01 Image Analysis.ipynb'
'notebooks/02 Captioning and dense captioning.ipynb'
'notebooks/03 Background removal.ipynb'


Analyze the 3 example notebooks for customized code analysis

In [5]:
root_dir = "notebooks"

# Loop through the folders
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        print(file)
        try:
            loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
            docs.extend(loader.load_and_split())
        except Exception as e:
            pass

02 Captioning and dense captioning.ipynb


KeyboardInterrupt: 

In [5]:
# Split into chunk of texts
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

We are going to load the settings from GitHub Codespace secrets instead

In [6]:
#load_dotenv("azure.env")

**Make sure this settings exist on your GitHub repository Codespace secrets!**


In my case both the model and deployment are named "text-embedding-ada-002"

>NOTE: it takes a few minutes to add the document embeddings to search (7 minutes for me)


In [6]:

# Initialize our embedding model
embeddings = OpenAIEmbeddings(
    deployment=os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME"),
    model=os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME"),
    openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
    openai_api_type="azure",
    chunk_size=1,
)

index_name = "index-pythonnotebooks"

# Set our Azure Search
acs = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_SEARCH_ADMIN_KEY"),
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)


In [ ]:

# Add documents to Azure Search
acs.add_documents(documents=texts)

In [12]:
api_key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
# Define Azure Cognitive Search as our retriever
retriever = AzureCognitiveSearchRetriever(
    content_key="content", top_k=10, index_name=index_name, api_key=api_key
    
)

In [26]:

# Set chatGPT 3.5 as our LLM
llm = AzureChatOpenAI(deployment_name=os.getenv("AZURE_OPENAI_MODEL_CHAT"), temperature=0.7, max_tokens=100, 
        openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"), 
        openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
        openai_api_version=os.getenv("AZURE_OPENAI_MODEL_CHAT_VERSION"),
        )

In [27]:
retriever

AzureCognitiveSearchRetriever(tags=None, metadata=None, service_name='ai050-search-unai', index_name='index-pythonnotebooks', api_key='91lSEwzCX8gCC1o43p6Xcwu5CLSzfMesZn3dL0s8fBAzSeA2zbVd', api_version='2020-06-30', aiosession=None, content_key='content', top_k=10)

In [28]:
llm

AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.7, model_kwargs={}, openai_api_key='3fa9294764c44064b4f2532d57cb9abc', openai_api_base='https://ai050-unai.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=100, tiktoken_model_name=None, deployment_name='gpt-35-turbo-unai', model_version='', openai_api_type='azure', openai_api_version='2023-03-15-preview')

## Testing

In [29]:
# Define a template message
template = """Use the following pieces of context to answer the question at the end. 
You are a python expert and you should demonstrate some python knowledge.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Set the Retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True,
)

In [30]:
questions = ["Could you explain the notebook 01 Image Analysis.ipynb"]

chat_history = []

for question in questions:
    result = qa_chain({"query": question, "chat_history": chat_history})
    # chat_history.append((question, result))
    print(f"Question: {question} \n")
    print(f"Answer: {result['result']} \n")
    print(
        f"Source: {json.loads(result['source_documents'][0].metadata['metadata'])['source']} \n"
    )

Question: Could you explain the notebook 01 Image Analysis.ipynb 

Answer: The notebook demonstrates how to use Azure Computer Vision 4.0 to analyze images and extract features such as tags, objects, captions, dense captions, and people. It also shows how to perform background removal and read text from an image. The notebook includes code examples and explanations of the features and their use cases. Thanks for asking! 

Source: notebooks/01 Image Analysis.ipynb 



In [25]:
questions = ["How to get image captions? Show me a python code"]

chat_history = []

for question in questions:
    result = qa_chain({"query": question, "chat_history": chat_history})
    # chat_history.append((question, result))
    print(f"Question: {question} \n")
    print(f"Answer: {result['result']} \n")
    print(
        f"Source: {json.loads(result['source_documents'][0].metadata['metadata'])['source']} \n"
    )

InvalidRequestError: The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.

In [14]:
questions = ["Explain the notebook 03 Background removal.ipynb"]

chat_history = []

for question in questions:
    result = qa_chain({"query": question, "chat_history": chat_history})
    # chat_history.append((question, result))
    print(f"Question: {question} \n")
    print(f"Answer: {result['result']} \n")
    print(
        f"Source: {json.loads(result['source_documents'][0].metadata['metadata'])['source']} \n"
    )

Question: Explain the notebook 03 Background removal.ipynb 

Answer: The notebook demonstrates how to use Azure Computer Vision 4 to remove the background from images. It provides functions to remove the background and get the alpha matte of the foreground object. The notebook also includes a batch processing example to remove the background from multiple images. Thanks for asking! 

Source: notebooks/03 Background removal.ipynb 



In [15]:
questions = ["How to remove background from an image using Azure AI?"]

chat_history = []

for question in questions:
    result = qa_chain({"query": question, "chat_history": chat_history})
    # chat_history.append((question, result))
    print(f"Question: {question} \n")
    print(f"Answer: {result['result']} \n")
    print(
        f"Source: {json.loads(result['source_documents'][0].metadata['metadata'])['source']} \n"
    )

Question: How to remove background from an image using Azure AI? 

Answer: To remove the background from an image using Azure AI, you can use the Azure Computer Vision service with the background removal feature. This feature can create an alpha matte that separates the foreground object from the background in an image. You can use the API endpoint and the appropriate headers to make a request to the service and get the edited image or the alpha matte.

Thanks for asking! 

Source: notebooks/03 Background removal.ipynb 



In [16]:
questions = ["How to get image captions?"]

chat_history = []

for question in questions:
    result = qa_chain({"query": question, "chat_history": chat_history})
    # chat_history.append((question, result))
    print(f"Question: {question} \n")
    print(f"Answer: {result['result']} \n")
    print(
        f"Source: {json.loads(result['source_documents'][0].metadata['metadata'])['source']} \n"
    )

Question: How to get image captions? 

Answer: To get image captions using Azure Computer Vision 4.0, you can use the "analyze" API with the "caption" feature. This will generate a caption that describes the content of the image. Make sure to include the necessary headers and endpoint URL when making the API call. Thanks for asking! 

Source: notebooks/02 Captioning and dense captioning.ipynb 

